In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q streamlit
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.8 MB/s eta 0:00:00

added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [ ]:
%%writefile app.py
import streamlit as st

st.title('My First Streamlit App')
st.write('Hello, *World!* :sunglasses:')

name = st.text_input('Enter your name')
if name:
    st.write(f'Hello, {name}!')

number = st.slider('Pick a number', 0, 100)
st.write(f'You selected: {number}')

Overwriting app.py


In [60]:
!streamlit run app.py &>/content/logs.txt &

In [61]:
import urllib
print("Password/Enpoint IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.125.179.169


In [ ]:
!npx localtunnel --port 8501

your url is: https://fruity-pandas-lick.loca.lt


In [3]:
!pip install pdfplumber -q
!pip install langchain_community -q
!pip install pypdf
!pip install chromadb -q
!pip install pinecone-client langchain openai pdfplumber
!pip install pinecone
!pip install tiktoken
!pip install langchain-openai
!pip install python-slugify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting require

In [ ]:
!pip install -U langchain-community
!pip install pdfplumber
!pip install chromadb
!pip install pinecone
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.0 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

import streamlit as st
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.embeddings.base import Embeddings
from typing import List
import os
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
from langchain.docstore import InMemoryDocstore
from langchain.document_loaders import PDFPlumberLoader

@st.cache_resource
def load_data(file_path):
    loader = PDFPlumberLoader(file_path)
    data = loader.load()
    return data

path_2023 = '/content/drive/MyDrive/Colab Notebooks/encore/project03/2023.pdf'

data = load_data(path_2023)

class DistilKoBERTEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = AutoModel.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = []
        for text in texts:
            inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
            # Remove token_type_ids from the inputs if it exists
            inputs = {key: value for key, value in inputs.items() if key != 'token_type_ids'}
            with torch.no_grad():
                outputs = self.model(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze(0).numpy()
            embeddings.append(embedding.tolist())
        return embeddings

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]


@st.cache_resource
def load_model():
    # 모델 로딩 및 초기화 코드
    model_name = 'monologg/distilkobert'
    embedding_model = DistilKoBERTEmbeddings(model_name=model_name)

    # FAISS 벡터 저장소 초기화
    dimension = 768
    index = faiss.IndexFlatL2(dimension)
    vector_db = FAISS(
        embedding_function=embedding_model,
        index=index,
        docstore=InMemoryDocstore({}),
        index_to_docstore_id={}
    )

    for row in data:
      if isinstance(row.page_content, str):
        vector_db.add_texts(
            texts=[row.page_content], # Enclose row.page_content in a list
            metadatas=[{'page': row.metadata['page']}],
            ids=[f"page_{row.metadata['page']}"]
        )

    # OpenAI 설정
    os.environ["OPENAI_API_KEY"] = ''
    llm = OpenAI(temperature=0.5, max_tokens=200)

    # 프롬프트 템플릿 설정
    prompt_template = """
    다음 정보를 바탕으로 질문에 답변하세요.

    {context}

    질문: {question}
    간단한 답변:"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    # RetrievalQA 체인 생성
    retriever = vector_db.as_retriever(search_kwargs={"k": 3})
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )

    return chain

def main():
    st.title("질문 답변 시스템")

    with st.spinner("모델을 로딩 중입니다. 잠시만 기다려주세요..."):
        chain = load_model()

    st.success("모델 로딩이 완료되었습니다. 아래 채팅창에서 질문을 입력하세요.")

    # Initialize chat history in session state
    if "messages" not in st.session_state:
        st.session_state.messages = []  # List to store chat messages

    # Display existing chat messages
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # Input box for the user to type their question
    if question := st.chat_input("질문을 입력하세요..."):
        # Add user's message to the chat history
        st.session_state.messages.append({"role": "user", "content": question})

        # Display user's message immediately
        with st.chat_message("user"):
            st.markdown(question)

        # Generate the response using the chain
        with st.spinner("답변을 생성 중입니다..."):
            result = chain({"query": question})
            answer = result['result']

        # Add the system's response to the chat history
        st.session_state.messages.append({"role": "assistant", "content": answer})

        # Display the system's response
        with st.chat_message("assistant"):
            st.markdown(answer)

        # Optionally, show referenced documents in expandable sections
        with st.chat_message("assistant"):
            st.markdown("**참고 문서:**")
            for doc in result['source_documents']:
                with st.expander(f"페이지 {doc.metadata['page']} 내용 보기"):
                    st.write(doc.page_content)

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
%%writefile app.py

import streamlit as st
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.embeddings.base import Embeddings
from typing import List
import os
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
from langchain.docstore import InMemoryDocstore
from langchain.document_loaders import PDFPlumberLoader
import tempfile

@st.cache_resource
def load_data(file_path):
    loader = PDFPlumberLoader(file_path)
    data = loader.load()
    return data

path_2023 = '/content/drive/MyDrive/Colab Notebooks/encore/project03/2023.pdf'


class DistilKoBERTEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = AutoModel.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = []
        for text in texts:
            inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
            # Remove token_type_ids from the inputs if it exists
            inputs = {key: value for key, value in inputs.items() if key != 'token_type_ids'}
            with torch.no_grad():
                outputs = self.model(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze(0).numpy()
            embeddings.append(embedding.tolist())
        return embeddings

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

def create_model(data):
    model_name = 'monologg/distilkobert'
    embedding_model = DistilKoBERTEmbeddings(model_name=model_name)

    dimension = 768
    index = faiss.IndexFlatL2(dimension)
    vector_db = FAISS(
        embedding_function=embedding_model,
        index=index,
        docstore=InMemoryDocstore({}),
        index_to_docstore_id={}
    )

    for row in data:
      if isinstance(row.page_content, str):
        vector_db.add_texts(
            texts=[row.page_content],
            metadatas=[{'page': row.metadata['page']}],
            ids=[f"page_{row.metadata['page']}"]
        )

    os.environ["OPENAI_API_KEY"] = ''
    llm = OpenAI(temperature=0.5, max_tokens=200)

    prompt_template = """
    다음 정보를 바탕으로 질문에 답변하세요.

    {context}

    질문: {question}
    간단한 답변:"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    retriever = vector_db.as_retriever(search_kwargs={"k": 3})
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )

    return chain

def main():
    st.title("질문 답변 시스템")

    # 세션 상태 초기화
    if 'model' not in st.session_state:
        default_data = load_data(path_2023)
        st.session_state.model = create_model(default_data)
        st.session_state.current_file = "기본 PDF"

    # 사이드바에 파일 업로더 추가
    uploaded_file = st.sidebar.file_uploader("PDF 파일을 업로드하세요", type="pdf")

    if uploaded_file is not None and uploaded_file.name != st.session_state.current_file:
        with st.spinner("새로운 PDF 파일을 처리 중입니다..."):
            with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp_file:
                tmp_file.write(uploaded_file.getvalue())
                tmp_file_path = tmp_file.name

            new_data = load_data(tmp_file_path)
            st.session_state.model = create_model(new_data)
            st.session_state.current_file = uploaded_file.name
            st.session_state.messages = []  # 채팅 기록 초기화
        st.success(f"'{uploaded_file.name}' 파일로 모델이 업데이트되었습니다.")

    st.sidebar.write(f"현재 사용 중인 파일: {st.session_state.current_file}")

    # 채팅 인터페이스
    if "messages" not in st.session_state:
        st.session_state.messages = []

    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    if question := st.chat_input("질문을 입력하세요..."):
        st.session_state.messages.append({"role": "user", "content": question})
        with st.chat_message("user"):
            st.markdown(question)

        with st.spinner("답변을 생성 중입니다..."):
            result = st.session_state.model({"query": question})
            answer = result['result']

        st.session_state.messages.append({"role": "assistant", "content": answer})
        with st.chat_message("assistant"):
            st.markdown(answer)

        with st.chat_message("assistant"):
            st.markdown("**참고 문서:**")
            for doc in result['source_documents']:
                with st.expander(f"페이지 {doc.metadata['page']} 내용 보기"):
                    st.write(doc.page_content)

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
%%writefile app.py

import streamlit as st
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter
from pinecone import Pinecone, ServerlessSpec
import os

@st.cache_resource
def load_data(file_path):
    loader = PDFPlumberLoader(file_path)
    data = loader.load()
    return data

@st.cache_resource
def load_model():
    # Pinecone API 키와 환경 설정
    api_key = ""
    environment = ""

    # Pinecone 클라이언트 초기화
    pc = Pinecone(api_key=api_key)
    index_name = "project3"

    # Pinecone 인덱스 생성
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=1536,
            metric="cosine",
            spec=ServerlessSpec(
                cloud="awp",
                region="us-east1"
            )
        )

    # Pinecone 인덱스 로드
    index = pc.Index(index_name)

    # OpenAI 임베딩 모델 설정
    os.environ["OPENAI_API_KEY"] = ""
    embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

    # 문서 로드 및 분할
    path_2023 = '/content/drive/MyDrive/Colab Notebooks/encore/project03/2023.pdf'
    data = load_data(path_2023)
    text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=50)
    split_texts = text_splitter.split_documents(data)

    # Pinecone에 문서 저장
    vectorstore = Pinecone.from_documents(split_texts, embedding_model, index_name=index_name)

    # RetrievalQA 체인 생성
    llm = OpenAI(temperature=0, model_name="gpt-4")
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever())

    return qa_chain

def main():
    st.title("질문 답변 시스템")

    with st.spinner("모델을 로딩 중입니다. 잠시만 기다려주세요..."):
        chain = load_model()

    st.success("모델 로딩이 완료되었습니다. 아래 채팅창에서 질문을 입력하세요.")

    if "messages" not in st.session_state:
        st.session_state.messages = []

    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    if question := st.chat_input("질문을 입력하세요..."):
        st.session_state.messages.append({"role": "user", "content": question})

        with st.chat_message("user"):
            st.markdown(question)

        with st.spinner("답변을 생성 중입니다..."):
            result = chain({"query": question})
            answer = result['result']

        st.session_state.messages.append({"role": "assistant", "content": answer})

        with st.chat_message("assistant"):
            st.markdown(answer)

        with st.chat_message("assistant"):
            st.markdown("**참고 문서:**")
            for doc in result['source_documents']:
                with st.expander(f"페이지 {doc.metadata['page']} 내용 보기"):
                    st.write(doc.page_content)

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
%%writefile app.py

import streamlit as st
import os
import time
import pandas as pd
import pdfplumber
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document  # 추가
from google.colab import files
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
from slugify import slugify

# Pinecone API 설정
api_key = ""
environment = ""
index_name = ""

# OpenAI API 키 설정
openai_api_key = ""
os.environ["OPENAI_API_KEY"] = openai_api_key

# Pinecone 클라이언트 초기화
pc = Pinecone(api_key=api_key)
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")  # 기본 OpenAI 임베딩 모델

# Pinecone 인덱스 생성
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # OpenAI text-embedding-ada-002 차원
        metric="cosine",
        spec=ServerlessSpec(
            cloud="awp",
            region="us-east1"
        )
    )

index = pc.Index(index_name)

# GPT-4 모델을 사용하는 함수
def generate_with_gpt4(prompt):
    try:
        prompt = prompt[:3000]
        response = OpenAI(api_key=openai_api_key).chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            n=2,
            temperature=0.7
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"GPT 생성 중 오류 발생: {e}")
        return None

# 파일 업로드 함수
def upload_file():
    uploaded = files.upload()  # 파일 업로드
    return list(uploaded.keys())[0]  # 업로드된 첫 번째 파일의 경로 반환

# 파일 형식 처리 함수
def load_file(uploaded_file):
    ext = uploaded_file.name.split('.')[-1].lower()  # 파일 확장자 추출
    content = ""

    if ext == 'pdf':
        with pdfplumber.open(uploaded_file) as pdf:
            for page in pdf.pages:
                content += page.extract_text()  # PDF에서 텍스트 추출
    elif ext == 'csv':
        df = pd.read_csv(uploaded_file)  # CSV 파일 읽기
        content = df.to_string()  # 텍스트로 변환
    elif ext == 'txt':
        content = uploaded_file.read().decode('utf-8')  # 텍스트 파일 읽기
    elif ext == 'xlsx':
        df = pd.read_excel(uploaded_file)  # 엑셀 파일 읽기
        content = df.to_string()  # 텍스트로 변환
    else:
        print(f"지원하지 않는 파일 형식입니다: {ext}")

    return content

# Pinecone에 이미 저장된 데이터 ID를 확인하는 함수
def get_existing_ids(index):
    existing_ids = []
    try:
        # Pinecone에서 데이터 100개를 가져오기
        fetch_response = index.fetch(ids=[f"chunk_{i}" for i in range(100)])
        existing_ids = [item.id for item in fetch_response['vectors'].values()]
    except Exception as e:
        print(f"데이터 확인 중 오류 발생: {e}")
    return existing_ids

# 업로드할 데이터가 이미 Pinecone에 있는지 확인하는 함수
def upload_data_if_not_exists(split_texts, index, file_name):
    # 기존 데이터 ID 가져오기
    existing_ids = get_existing_ids(index)

    for i, doc in enumerate(split_texts):
        # 파일명과 번호를 결합한 ID 생성
        chunk_id = f"{file_name}_{i}"

        if chunk_id not in existing_ids:
            try:
                time.sleep(0.1)  # Rate Limiting 방지
                vector = embedding_model.embed_query(doc.page_content)  # 텍스트 임베딩 생성
                index.upsert([(chunk_id, vector, {"source": chunk_id, "text": doc.page_content})])  # Pinecone에 업로드
                print(f"업로드된 ID: {chunk_id}")
            except Exception as e:
                print(f"chunk_{i}에서 오류 발생: {e}")
        else:
            print(f"ID {chunk_id}는 이미 Pinecone에 존재합니다. 업로드를 건너뜁니다.")

# 텍스트 분할 함수
def split_text(content):
    text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=50)

    # 문자열을 Document 객체로 변환하여 반환
    documents = text_splitter.split_text(content)
    return [Document(page_content=doc) for doc in documents]  # Document 객체로 반환

# Pinecone에서 쿼리를 검색하는 함수
def search_in_pinecone(query, index, embedding_model, top_k=3):
    try:
        # 쿼리 임베딩 생성
        query_vector = embedding_model.embed_query(query)

        # Pinecone 인덱스에서 검색
        results = index.query(
            vector=query_vector,
            top_k=top_k,
            include_metadata=True
        )
        return results
    except Exception as e:
        print(f"검색 중 오류 발생: {e}")
        return None

def main():
    st.title("문서 Q&A 시스템")

    # 채팅 기록 초기화
    if "messages" not in st.session_state:
        st.session_state.messages = []

    # 채팅 기록 표시
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # 사용자 입력 받기
    if prompt := st.chat_input("질문을 입력하세요:"):
        # 사용자 메시지 추가
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        # Pinecone에서 검색된 결과
        search_results = search_in_pinecone(prompt, index, embedding_model)

        if search_results:
            # 검색된 텍스트를 결합하여 GPT-4에게 전달할 프롬프트 만들기
            context = "\n".join([match.metadata['text'] for match in search_results.matches])
            gpt_prompt = f"다음 내용을 바탕으로 사용자의 질문에 답해주세요: {prompt}\n\n{context}"

            # GPT-4 모델을 사용해 답변 생성
            with st.chat_message("assistant"):
                with st.spinner("답변을 생성 중입니다..."):
                    response_gpt4 = generate_with_gpt4(gpt_prompt)
                    if response_gpt4:
                        st.markdown(response_gpt4)
                        # 어시스턴트 메시지 추가
                        st.session_state.messages.append({"role": "assistant", "content": response_gpt4})

                        st.markdown("### 참고한 페이지:")
                        for i, match in enumerate(search_results.matches[:3]):  # 상위 3개 결과만 출력

                            # 전체 텍스트를 토글 형식으로 표시
                            with st.expander(f"내용 보기 ({i + 1})"):
                              # 텍스트 앞부분만 가져오기 (최대 200자)
                                preview_text = match.metadata.get('text', '')[:400]
                                source = match.metadata.get('source', '알 수 없음')  # 소스 정보 가져오기
                                st.markdown(f"**출처:** {source}")
                                st.markdown(f"{preview_text}...")  # 앞부분만 표시
                    else:
                        st.error("답변 생성 중 오류가 발생했습니다.")
        else:
            with st.chat_message("assistant"):
                st.warning("검색 결과가 없습니다.")
                st.session_state.messages.append({"role": "assistant", "content": "검색 결과가 없습니다."})

    # 파일 업로드 섹션
    st.sidebar.header("파일 업로드")
    uploaded_file = st.sidebar.file_uploader("파일을 업로드하세요", type=['pdf', 'csv', 'txt', 'xlsx'])

    if uploaded_file is not None:
        st.sidebar.write(f"업로드된 파일: {uploaded_file.name}")

        file_name = os.path.splitext(uploaded_file.name)[0]
        file_name = slugify(file_name)

        st.sidebar.write(f"영문 파일명: {file_name}")

        if st.sidebar.button("파일 처리 및 업로드"):
            with st.spinner("파일을 처리 중입니다..."):
                file_content = load_file(uploaded_file)
                split_texts = split_text(file_content)
                upload_data_if_not_exists(split_texts, index, file_name)
            st.sidebar.success("파일 처리 및 업로드가 완료되었습니다!")

if __name__ == "__main__":
    main()


Overwriting app.py
